# Task 3 - Pattern matching
Karen from the security department asked you for help again. Now she wants to find suspicious withdrawals from ATMs. In some ATMs, clients are able to withdraw up to 100PLN without the PIN. The security specialists consider that 3 withdrawals within 10 minutes without PIN should block the card. Please help her with this task.

## Project initialization

In [ ]:
%reload_ext streaming_jupyter_integrations.magics
%flink_connect

## Data definition
You have access to Kafka topic `atm` with data about ATM withdrawals. First, you have to prepare definitions for this table. Sample messages are available in `data/` directory.

In [ ]:
%%flink_execute_sql
CREATE TABLE atm (
    user_id INT,
    amount INT,
    atm_id STRING,
    ts TIMESTAMP(3),
    WATERMARK FOR ts AS ts
) WITH (
    'connector' = 'filesystem',
    'path' = 'file:///home/jovyan/task3/data',
    'format' = 'json'
)

## Pattern recognition
Now create SELECT clause with pattern recognition. In the output we want to have `user_id`, the withdrawn amount and the id of the last ATM. Details about Pattern Recognition you can find [HERE](https://nightlies.apache.org/flink/flink-docs-master/docs/dev/table/sql/queries/match_recognize/).

In [ ]:
%%flink_execute_sql
SELECT T.user_id, T.last_atm, T.amount
FROM atm
    MATCH_RECOGNIZE (
      PARTITION BY user_id
      ORDER BY ts
      MEASURES
        LAST(A.atm_id) AS last_atm,
        SUM(amount) AS amount
      PATTERN (A{3}) WITHIN INTERVAL '10' MINUTE
      DEFINE
        A AS amount <= 100
    ) AS T


## Write to Kafka
To use the power of Flink in a real case, you need to send the data to the relevant kafka topic - in this case topic `task3_output`. The messages should be in JSON format and have 4 fields: 
- `user_id`
- `last_atm_id`
- `withdrawn_amount`
- `author` - your name

In [ ]:
%%flink_execute_sql
CREATE TABLE task3_output (
    user_id INT,
    withdrawn_amount INT,
    last_atm_id STRING,
    author STRING
) WITH (
    'connector' = 'kafka',
    'topic' = 'task3_output',
    'properties.bootstrap.servers' = '172.18.0.3:9092',
    'properties.group.id' = 'task3',
    'scan.startup.mode' = 'earliest-offset',
    'format' = 'json'
)

In [ ]:
%%flink_execute_sql
INSERT INTO task3_output
    SELECT
      T.user_id,
      T.amount,
      T.last_atm,
      'AUR'
    FROM atm
        MATCH_RECOGNIZE (
          PARTITION BY user_id
          ORDER BY ts
          MEASURES
            LAST(A.atm_id) AS last_atm,
            SUM(amount) AS amount
          PATTERN (A{3}) WITHIN INTERVAL '10' MINUTE
          DEFINE
            A AS amount <= 100
        ) AS T